In [1]:
cd ../..

e:\GitHub\computervision


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [3]:
# resnext101_32x8d
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [4]:
data_dir = 'E:\Datasets\cor'
data_folders = ['train', 'val']

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in data_folders}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in data_folders}

dataset_sizes = {x: len(image_datasets[x]) for x in data_folders}

class_names = image_datasets['train'].classes
print(class_names)

['normal', 'rain']


In [5]:
working_folder = 'notebooks/resnext'
models_folder = os.path.join(working_folder, 'models')
from datetime import datetime

curr_time = datetime.now()
# curr_time = datetime(curr_time.year, curr_time.month, curr_time.day, curr_time.hour)
curr_time = curr_time.strftime("%m-%d-%Y %H-%M-%S")

output_folder = os.path.join(models_folder, curr_time)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [7]:
def imshow(inp, title):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.title(title)
    plt.show()

In [8]:
def train_model(model, criterion, optimizer, scheduler, output_folder, start_epoch=1, num_epochs=25, epoch_save_interval=2):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    log_path = f"{output_folder}/log.txt"
    
    if not os.path.exists(log_path): 
        log = open(log_path,'x')
        log.close()
    
    for epoch in range(start_epoch, num_epochs+1):
        epoch_info = 'Epoch {}/{}'.format(epoch, num_epochs)
        
        with open(log_path,'a') as log:
            log.writelines('\n'+epoch_info+'\n')
            log.writelines('-' * 10+'\n')
        
        print(epoch_info)
        print('-' * 10)
        

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            loss_acc_info = '{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc)
            with open(log_path,'a') as log:
                log.writelines(loss_acc_info+'\n')
            print(loss_acc_info)
            
            if epoch % epoch_save_interval == 0:
                torch.save(model.state_dict(), f'{output_folder}/epoch_{epoch}.pth')
                
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
    
    torch.save(model.state_dict(), f'{output_folder}/last.pth')
    
    time_elapsed = time.time() - since
    
    complete_text = 'Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)
    print(complete_text)
    best_acc_text = 'Best val Acc: {:4f}'.format(best_acc)
    print(best_acc_text)
    
    with open(log_path,'a') as log:
        log.writelines(complete_text+'\n')
        log.writelines(best_acc_text+'\n')
        
    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), f'{output_folder}/best.pth')
    
    return model

In [9]:
continue_training = True

In [10]:
#### Finetuning the convnet ####
# Load a pretrained model and reset final fully connected layer.

model = models.resnext101_32x8d(pretrained=True)
for param in model.parameters(): # Turning parameters not trainable
    param.requires_grad = False

# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
num_ftrs = model.fc.in_features    
model.fc = nn.Linear(num_ftrs, 2) # A newly defined layer is created with requires_grad=True by default
    
if continue_training:
    output_folder = os.path.join(models_folder, '12-05-2023 08-29-48')
    model.load_state_dict(torch.load(os.path.join(output_folder, 'epoch_14.pth')))

if not os.path.exists(output_folder): 
    os.makedirs(output_folder)
        
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Only the fully connected layer is being optimized
optimizer = optim.SGD(model.fc.parameters(), lr=0.001)
# StepLR Decays the learning rate of each parameter group by gamma every step_size epochs
# Decay LR by a factor of 0.1 every 7 epochs
# Learning rate scheduling should be applied after optimizer’s update
# e.g., you should write your code this way:
# for epoch in range(100):
#     train(...)
#     validate(...)
#     scheduler.step()

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, step_lr_scheduler, output_folder, start_epoch=15, num_epochs=25)

c:\Users\Adm\anaconda3\envs\cv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Adm\anaconda3\envs\cv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt101_32X8D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt101_32X8D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 15/25
----------
train Loss: 0.1048 Acc: 0.9636
val Loss: 0.5803 Acc: 0.7286

Epoch 16/25
----------
train Loss: 0.1048 Acc: 0.9636
val Loss: 0.6745 Acc: 0.6777

Epoch 17/25
----------
train Loss: 0.1043 Acc: 0.9637
val Loss: 0.7605 Acc: 0.6422

Epoch 18/25
----------
train Loss: 0.1005 Acc: 0.9658
val Loss: 0.5295 Acc: 0.7459

Epoch 19/25
----------


KeyboardInterrupt: 